In [3]:
from functools import partial # for trials
import numpy as np # for accuracy math
import os # for paths
import torch # for nn instantiation
import torch.nn as nn # for nn objects
import torch.nn.functional as F # for forward method
import torch.optim as optim # for optimization
from torch.utils.data import random_split # for train/test split
import torchvision # for data transforms
import torchvision.transforms as transforms # for transform methods
import ray
from ray import tune # for trialing
# from ray.tune import CLIReporter # for trial reporting
from ray.tune import JupyterNotebookReporter # for trial reporting
from ray.tune.integration.torch import is_distributed_trainable
from torch.nn.parallel import DistributedDataParallel
from ray.tune.integration.torch import DistributedTrainableCreator
from ray.tune.integration.torch import distributed_checkpoint_dir
from ray.tune.schedulers import ASHAScheduler # for trial scheduling
# from ray.tune.schedulers import HyperBandForBOHB # for trial scheduling
# from ray.tune.suggest.bohb import TuneBOHB # for trial selection/pruning
# from ray.tune.suggest.dragonfly import DragonflySearch
# from dragonfly.opt.gp_bandit import CPGPBandit
from ray.tune.schedulers import AsyncHyperBandScheduler
# from dragonfly import load_config
# from dragonfly.exd.experiment_caller import CPFunctionCaller, EuclideanFunctionCaller
from ray.tune.suggest.bayesopt import BayesOptSearch

import GPy
import sklearn
from ray.tune.schedulers import pb2
from ray.tune.schedulers.pb2 import PB2

from ray.tune.suggest import ConcurrencyLimiter

import ConfigSpace as CS # for configuration bounds
from collections import OrderedDict # for dynamic configuration definition
from pathlib import Path # for OS agnostic path definition

# import itertools package 
import itertools 
from itertools import combinations, combinations_with_replacement
from itertools import product

import math
import pandas as pd

# allow configuration copying
from copy import deepcopy

import optuna
# from optuna.samplers import TPESampler
from optuna.multi_objective.samplers import MOTPEMultiObjectiveSampler
# from optuna.pruners import SuccessiveHalvingPruner

# from ray.tune.schedulers.pb2_utils import normalize, optimize_acq, select_length, UCB, standardize, TV_SquaredExp

In [4]:
# # set data and checkpoint locations
# p = Path('.')
# d = p / 'data'
# r = p / 'ray_results'
# l = p / 'checkpoints' / 'layers'
# n = p / 'checkpoints' / 'layers'

# # set computation location(s)
# gpus = torch.cuda.device_count()
# device = "cuda:0" if gpus else "cpu"

# # set number or fraction of processing units (per training worker) you'd like to utilize, if any at all
# # cpu_use must be grater than zero
# cpu_use = 1 if gpus else 0.5
# gpu_use = 0.25 if gpus else 0

# # set experiment hyperparameters
# num_samples = 2 ** (6 if gpus else 4)
# max_time = 10 * (4 if gpus else 1)
# gpus_per_trial = 0.5 if gpus else 0

# set data and checkpoint locations
p = Path('.')
dt = p / 'data'
r = p / 'ray_results'
l = p / 'checkpoints' / 'layers'
n = p / 'checkpoints' / 'layers'

# set computation location(s)
cpus = os.cpu_count()
gpus = torch.cuda.device_count()

# set number or fraction of processing units (per training worker) you'd like to utilize, if any at all
# cpu_use must be grater than zero
max_concurrent_trials = cpus#int(cpus/2) if cpus > 1 else cpus
cpu_use = 2#2 / gpus if gpus else 1
gpu_use = 0.5#gpus/max_concurrent_trials if gpus else 0

# set experiment hyperparameters
# oom = 2 if gpus else 3 # order of magnitude
num_samples = 2 / gpu_use#** oom
max_time = 10# * oom

Since the neuron configuration we want is dependent upon the number of layers we have, we need to work flatten the feature space a bit. We can reduce the high-dminesional setups to a slightly less high-dminesional string of base-n nodes.

In [ ]:
# define feature space for hashing

# get max number of respective layers (exclusive)
excl_conv_n_max = 5
excl_full_n_max = 5

# get number of respective layer attributes (max exclusive)
c_min = 3**2 
c_max = 3**5
k_min = 2
k_max = 5
m_min = 0
m_max = 1
f_min = 2**2
f_max = 2**6
d_min = 5
d_max = 11

c = c_max - c_min # convolutional layer options
k = k_max - k_min # kernel options
m = m_max - m_min # max pool layer options
f = f_max - f_min # fully connected layer options
d = d_max - d_min # dropout layer options

# conv = set(range(c_max)) - set(range(c_min))
# full = set(range(f_max)) - set(range(f_min))
conv = list(range(c_max)[c_min:])
kern = list(range(k_max)[k_min:])
maxp = list(range(m_max)[m_min:])
full = list(range(f_max)[f_min:])
drop = list(range(d_max)[d_min:])

# c_comb = list(combinations_with_replacement(conv,2))
c_comb = []
k_comb = []
m_comb = []
f_comb = []
d_comb = []

for layers in range(1,4):
    cm = list(product(conv,kern,maxp,repeat=1))
    print(cm)
    c_comb += list(combinations_with_replacement(cm,layers))
# print(conv)
# print(maxp)
# print(c_comb)
# pd.DataFrame(c_comb).to_csv("c_comb.csv")
for layers in range(1,2):
    m_comb += list(combinations_with_replacement(maxp,layers))
for layers in range(1,5):
    fd = list(product(full,drop))
    f_comb += list(combinations_with_replacement(fd,layers))
# print(full)
# print(drop)
# print(f_comb)
# pd.DataFrame(f_comb).to_csv("f_comb.csv")
for layers in range(1,2):
    d_comb += list(combinations_with_replacement(drop,layers))
#     print("Fully connected layer %s range: %s" % (layers,len(f_comb)) )
#     print("\n")
# [pd.DataFrame(comb).to_csv("%s.csv" % name) for name,comb in zip(["c_comb","m_comb","f_comb","d_comb"],[c_comb,m_comb,f_comb,d_comb])]

[(9, 2, 0), (9, 3, 0), (9, 4, 0), (10, 2, 0), (10, 3, 0), (10, 4, 0), (11, 2, 0), (11, 3, 0), (11, 4, 0), (12, 2, 0), (12, 3, 0), (12, 4, 0), (13, 2, 0), (13, 3, 0), (13, 4, 0), (14, 2, 0), (14, 3, 0), (14, 4, 0), (15, 2, 0), (15, 3, 0), (15, 4, 0), (16, 2, 0), (16, 3, 0), (16, 4, 0), (17, 2, 0), (17, 3, 0), (17, 4, 0), (18, 2, 0), (18, 3, 0), (18, 4, 0), (19, 2, 0), (19, 3, 0), (19, 4, 0), (20, 2, 0), (20, 3, 0), (20, 4, 0), (21, 2, 0), (21, 3, 0), (21, 4, 0), (22, 2, 0), (22, 3, 0), (22, 4, 0), (23, 2, 0), (23, 3, 0), (23, 4, 0), (24, 2, 0), (24, 3, 0), (24, 4, 0), (25, 2, 0), (25, 3, 0), (25, 4, 0), (26, 2, 0), (26, 3, 0), (26, 4, 0), (27, 2, 0), (27, 3, 0), (27, 4, 0), (28, 2, 0), (28, 3, 0), (28, 4, 0), (29, 2, 0), (29, 3, 0), (29, 4, 0), (30, 2, 0), (30, 3, 0), (30, 4, 0), (31, 2, 0), (31, 3, 0), (31, 4, 0), (32, 2, 0), (32, 3, 0), (32, 4, 0), (33, 2, 0), (33, 3, 0), (33, 4, 0), (34, 2, 0), (34, 3, 0), (34, 4, 0), (35, 2, 0), (35, 3, 0), (35, 4, 0), (36, 2, 0), (36, 3, 0), (36, 4

In [1]:
# trial value builder
def buildtrial(conv_kern=rang,conv_out)

SyntaxError: invalid syntax (<ipython-input-1-fdb8ab86880e>, line 1)

In [ ]:
# for conversion from dec to whatever we end up using
# most to least significant digit
def numberToBase(n, b):
    if n == 0:
        return [0]
    digits = []
    while n:
        digits.append(int(n % b))
        n //= b
    rev = digits[::-1]
    return rev

def feature_spacing():
    
    # create empty list to store the 
    # combinations 
#     conv_combinations = list(combinations([c_comb,m_comb],2))
#     print("np.shape(conv_combinations[0])",np.shape(conv_combinations[0]))
# #     pd.DataFrame(conv_combinations).to_csv('conv_combinations.csv')
#     full_combinations = list(combinations([f_comb,d_comb],2))
#     print("np.shape(full_combinations[0])",np.shape(full_combinations[0]))
#     pd.DataFrame(full_combinations).to_csv('full_combinations.csv')
    unique_combinations = list(product(c_comb,f_comb))
    print("np.shape(unique_combinations)",np.shape(unique_combinations))
#     pd.DataFrame(unique_combinations).to_csv('unique_combinations.csv')
#     [pd.DataFrame(comb).to_csv("%s.csv" % name) for name,comb in zip(["conv_combinations","full_combinations","unique_combinations"],[conv_combinations,full_combinations,unique_combinations])]
    total_uniques = len(unique_combinations)
    total_points = total_uniques**2
    total_cvs = len(c_comb)
    total_krn = len(k_comb)
    total_mxp = len(m_comb)
    total_fcs = len(f_comb)
    total_drp = len(d_comb)
    
    columns = ["base","nodes_req","sparcity","sparcity_pcnt","denoise_pcnt"]
    values = [1,total_uniques,total_points - total_uniques,(total_points - total_uniques) / total_points,0]
    
    cf = []
    
    for layer in [total_cvs,total_mxp,total_fcs,total_drp]:#,total_uniques]:
        results = {
            "base": [1],
            "nodes_req": [total_uniques],
            "sparcity": [total_points - total_uniques],
            "max_necc_base_value":[0],
            "nodes+_req": [0],
            "subsparcity": [0],
            "unexplained":[0],
            "sparcity_pcnt": [(total_points - total_uniques) / total_points * 100],
            "subsparcity_pcnt": [0],
            "denoise_pcnt":[0],
            "complexity":[0]
        }

        report = pd.DataFrame(results)
    
        for base in range(2,101):
            results["base"] = [base]
            results["nodes_req"] = [math.ceil(math.log(layer,(base)))]
            results["nodes+_req"] = [math.floor(math.log(layer,(base)))]
            
            results["sparcity"] = [base**math.ceil(math.log(layer,base)) - layer]
            results["subsparcity"] = [-(base**math.floor(math.log(layer,base)) - layer)]
            
            results["sparcity_pcnt"] = [(base**math.ceil(math.log(layer,(base))) - base**math.log(layer,(base)))/(base**math.ceil(math.log(layer,(base))))*100]
            results["subsparcity_pcnt"] = [-((base**math.floor(math.log(layer,(base))) - base**math.log(layer,(base)))/(base**math.floor(math.log(layer,(base))))*100)]
            
#             results["max_necc_base_value"] = [numberToBase((results["base"][0]**results["nodes+_req"][0]+results["subsparcity"][0]),results["base"][0])]
            results["max_necc_base_value"] = [numberToBase(layer,base)]
            results["unexplained"] = [(-(base**math.floor(math.log(layer,base)) - layer))*(math.floor(math.log(layer,(base))))]
            
            results["denoise_pcnt"] = [math.floor(((total_points-(math.ceil(math.log(layer,base)))**2)/total_points)*100)]
        
            results["complexity"] = [results["nodes_req"][0]*(results["sparcity"][0]+1)]

            report = report.append(pd.DataFrame(results))
            
            
        report.index = [x for x in range(1, len(report.values)+1)]
        report.drop([1],axis=0,inplace=True)
        report.sort_values(["nodes+_req","sparcity","unexplained","subsparcity","sparcity_pcnt","base"],inplace=True)
        
        cf.append(report.iloc[0])
    
    return cf

bases = feature_spacing()
[print(r,"\n") for r in bases]

base_c = bases[0]["base"]
base_m = bases[1]["base"]
base_f = bases[2]["base"]
base_d = bases[3]["base"]

In [ ]:
print("For the convolutional layers, base %s seems to allow us to use: \n1.) the fewest nodes of value 2-100 (inclusive) with \n2.) the lowest number of invalid configuration indices (sparcity)." % bases[0]["base"])
print("For the max pooling layers, base %s seems to allow us to use: \n1.) the fewest nodes of value 2-100 (inclusive) with \n2.) the lowest number of invalid configuration indices (sparcity)." % bases[1]["base"])
print("For the linear layers, base %s seems to allow us to use: \n1.) the fewest nodes of value 2-100 (inclusive) with \n2.) the lowest number of invalid configuration indices (sparcity)." % bases[2]["base"])
print("For the dropout layers, base %s seems to allow us to use: \n1.) the fewest nodes of value 2-100 (inclusive) with \n2.) the lowest number of invalid configuration indices (sparcity)." % bases[3]["base"])

# print("We can use the ")

In [ ]:
def base_to_dec(num_list, base):
    num_list = num_list[::-1]
    num = 0
    for k in range(len(num_list)):
        dig = num_list[k]
        dig = int(dig)
        num += dig*(base**k)
    return num

def encode(config=[(24, 64),(0, 1),(13, 41),(0, 6)]):
    iconv = c_comb.index(config[0])
    imxp = m_comb.index(config[1])
    ifull = f_comb.index(config[2])
    dconv = d_comb.index(config[3])
    
    conv_hash = numberToBase(iconv,base_c)
    maxp_hash = numberToBase(imaxp,base_m)
    full_hash = numberToBase(ifull,base_f)
    drop_hash = numberToBase(idrop,base_d)
    
    return [conv_hash,maxp_hash,full_hash,drop_hash]

def decode(hash=([1, 7, 5, 0], [1, 0, 1, 0], [2, 9, 7], [6, 9, 7])):
    conv = base_to_dec(hash[0], base_c)
    maxp = base_to_dec(hash[1], base_m)
    full = base_to_dec(hash[2], base_f)
    drop = base_to_dec(hash[3], base_d)

    
    return [c_comb[conv],m_comb[maxp],f_comb[full],d_comb[drop]]

In [ ]:
# move data into sets for loading
def load_data(data_dir=dt.absolute()):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset,testset = [torchvision.datasets.CIFAR10(root=data_dir, train=is_train, download=True, transform=transform) for is_train in [True,False]]

    return trainset, testset

In [ ]:
# dynamically-generated nn that takes a 3-channel image and outputs a label
class Net(nn.Module):
    def __init__(self, hidden_layers=[[6, 16],[0, 1],[120,84],[0, 6]]):
        super(Net, self).__init__()
        hidden_convs,hidden_maxps,hidden_fcs,hidden_drops = hidden_layers
#         print(hidden_convs)
#         print(hidden_fcs)
# #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#         uf_input = 0
#         layer_list = OrderedDict()
        
#         layers = []
#         pre_flat_size = 32
#         in_channels = 3
#         out_kernel = None
        
#         layer_list['conv1'] = nn.Conv2d(3, hidden_convs[0], 5)
#         layer_list['pool1'] = nn.MaxPool2d(2, 2)

#         layer_input = layer_list['conv1'].out_channels
        
#         for layer_num, channels in enumerate(hidden_convs[1:], 2):
#             layer_list["conv%s" % layer_num]  = nn.Conv2d(layer_input, channels, 5)
#             layer_list["pool%s" % layer_num] = nn.MaxPool2d(2, 2)
#             layer_input = layer_list["conv%s" % layer_num].out_channels
        
        
#         layer_list["flat"] = nn.Flatten()
        
#         layer_list['fc1'] = nn.Linear(layer_input*5*5, hidden_fcs[0])
#         layer_list["relu1"]  = nn.ReLU()
        
#         layer_input = layer_list['fc1'].out_features
#         for (layer_num, features) in enumerate(hidden_fcs[1:], 2):
#             layer_list["fc%s" % layer_num]  = nn.Linear(layer_input, features)
#             layer_list["relu%s" % layer_num]  = nn.ReLU()
#             layer_input = layer_list["fc%s" % layer_num].out_features
            
        
#         layer_list['fco'] = nn.Linear(hidden_fcs[-1], 10)
    
#         self.layers = nn.Sequential(layer_list)
        
# #         print("New model: %s" % hidden_layers)
#     def forward(self, x):
#         x = self.layers(x)
#         return x
# #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
        layers = []
        pre_flat_size = 32
        in_channels = 3
        out_kernel = None

        for i in range(hidden_convs):
            if pre_flat_size > 7:
                out_channels = hidden_convs[i]
                kernel_size = 3
                layers.append(nn.Conv2d(in_channels, out_channels, kernel_size))
                pre_flat_size = pre_flat_size - kernel_size+1
    #             print("post conv: ",pre_flat_size)
                if hidden_maxps[i] & pre_flat_size > 3:
                    layers.append(nn.MaxPool2d(2, 2))
                    pre_flat_size = int(pre_flat_size / 2)
    #                 print("post pool: ",pre_flat_size)
                layers.append(nn.BatchNorm2d(out_channels))

            in_channels = out_channels
            out_kernel = kernel_size

        layers.append(nn.Flatten())

    #     self.flattening = nn.Sequential(*layers)

    #     layers = []
    #         print("pre_flat_size:",pre_flat_size)
        in_features = in_channels * pre_flat_size**2
        for i in range(hidden_fcs):
            out_features = hidden_fcs[i]
            layers.append(nn.Linear(in_features, out_features))
            layers.append(nn.ReLU())
            if hidden_drops[0] % 10:
                p = (hidden_drops[0] % 10) / 10
                layers.append(nn.Dropout(p))
            layers.append(nn.LayerNorm(out_features))

            in_features = out_features

        layers.append(nn.Linear(in_features, 10))
        layers.append(nn.LogSoftmax(dim=1))

    #     self.linearizing = nn.Sequential(*layers)
        return nn.Sequential(*layers)

In [ ]:
# train nn on data
def train_cifar(neuron_config, checkpoint_dir=None):
    print(neuron_config["conv_adjusted"])
    print(neuron_config["full_adjusted"])
    
    data_dir=dt.absolute()
    
    def cv_discrim(s): return 'conv_subindex_' in s
    def fc_discrim(s): return 'full_subindex_' in s
    cvs = [neuron_config[hp] for hp in list(filter(cv_discrim, neuron_config.keys()))]
    fcs = [neuron_config[hp] for hp in list(filter(fc_discrim, neuron_config.keys()))]
    
#     cfg = decode([cvs, fcs])
    cfg = decode([neuron_config["conv_adjusted"], neuron_config["full_adjusted"]])    
    net = Net(cfg)
    
    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=neuron_config["lr"], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data()

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader,valloader = [torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(neuron_config["batch_size"]),
        shuffle=True,
        num_workers=1) for subset in [train_subset,val_subset]]

    for epoch in range(neuron_config["epochs"]):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("Model: %s, Epoch: %d, Mini-batch: %5d, Loss: %.3f" % (cfg,epoch + 1, i + 1, running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=(correct / total))
        
    print("Finished Training")

In [ ]:
# get accuracy score
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=1)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [ ]:
#determine configuration boundary for nn based on number of layers
nodes_c = bases[0]["nodes_req"]
nodes_m = bases[1]["nodes_req"]
nodes_f = bases[2]["nodes_req"]
nodes_d = bases[3]["nodes_req"]
max_c = bases[0]["max_necc_base_value"]
max_m = bases[1]["max_necc_base_value"]
max_f = bases[2]["max_necc_base_value"]
max_d = bases[3]["max_necc_base_value"]

In [ ]:
# def configure_neurons(num_convs,num_fcs):
def configure_neurons():
    lr = {
        "name":"learning rate {10^(-⌊x⌋)",
#         "bounds":[1e-4,1e-1]
        "bounds":[x for x in range(1,4)]
    }
    batch_size = {
        "name":"batch size {2^⌊x⌋}",
        "bounds":[x for x in range(5,9)]
#         "bounds":[x for x in range(4,6)]
    }
    epochs = {
        "name":"epochs {10⌊x⌋}",
#         "bounds":[x for x in range(2,6)]
        "bounds":[1,2]
    }
    config_space = CS.ConfigurationSpace()
    config_space_dict = {"name": "neurons"}
    config_space_ray = {"name": "neurons"}
    
#     #start ConfigSpace API
#     config_space.add_hyperparameter(
#         CS.UniformFloatHyperparameter(name="lr", lower=1e-4, upper=1e-1, log=True))
#     config_space.add_hyperparameter(
#         CS.CategoricalHyperparameter(name="batch_size", choices=[2**x for x in range(2,6)]))
#     config_space.add_hyperparameter(
#         CS.CategoricalHyperparameter(name="epochs", choices=[10*x for x in range(2,5)]))
    
    #start ConfigSpace API
    config_space.add_hyperparameter(
        CS.UniformFloatHyperparameter(
            lr["name"],
            lr["bounds"][0],
            lr["bounds"][-1],
            log=True
        ))
    config_space.add_hyperparameter(
        CS.CategoricalHyperparameter(
            batch_size["name"], 
            batch_size["bounds"]
        ))
    config_space.add_hyperparameter(
        CS.CategoricalHyperparameter(
            epochs["name"], 
            epochs["bounds"]
        ))
    
#     #start Ray Search Space API
#     config_space_dict["lr"] = tune.loguniform(lower=1e-4, upper=1e-1)
#     config_space_dict["batch_size"] = tune.choice(categories=[2**x for x in range(2,6)])
#     config_space_dict["epochs"] = tune.choice(categories=[10*x for x in range(2,5)])
    
    #start Ray Search Space API
    config_space_ray[lr["name"]] = tune.loguniform(lr["bounds"][0],lr["bounds"][-1])
    config_space_ray[batch_size["name"]] = tune.choice(batch_size["bounds"])
    config_space_ray[epochs["name"]] = tune.choice(categories=epochs["bounds"])
    
    config_space_dict["conv_subindices"],config_space_dict["full_subindices"] = [],[]
    config_space_dict["conv_cases"],config_space_dict["full_cases"] = [],[]
    config_space_dict["conv_auxilliary"],config_space_dict["full_auxilliary"] = [],[]        
    config_space_dict["conv_adjusted"],config_space_dict["full_adjusted"] = [],[]
    config_space_dict["maxp_subindices"],config_space_dict["drop_subindices"] = [],[]
    config_space_dict["maxp_cases"],config_space_dict["drop_cases"] = [],[]
    config_space_dict["maxp_auxilliary"],config_space_dict["drop_auxilliary"] = [],[]        
    config_space_dict["maxp_adjusted"],config_space_dict["drop_adjusted"] = [],[]
    
    conv_lims,full_lims = [],[]
    maxp_lims,drop_lims = [],[]
    
    for subindex in range(nodes_c):
        # define hyperparameter reference attributes
        rule_name = "conv_subindex_%s" % subindex
        
        conv_rule = CS.UniformIntegerHyperparameter(rule_name, lower=0, upper=base_c-1, default_value=subindex%(base_c-1))
#         conv_dict_rule = tune.randint(lower=0, upper=base_c-1)
        
        # add hyperparameter to collections
        config_space.add_hyperparameter(conv_rule)
#         config_space_dict[rule_name] = conv_dict_rule
#         config_space_dict["conv_subindices"].append(conv_dict_rule)
    
        conv_rules = list(filter(lambda hp: "conv_subindex_" in hp.name, config_space.get_hyperparameters()))
        conv_dict_rules_keys = list(filter(lambda name: "conv_subindex_" in name, config_space_dict.keys()))
#         conv_dict_rules = {k:config_space_dict[k] for k in conv_dict_rules_keys if k in config_space_dict}
    
        # build banlist from collections
        rl = deepcopy(config_space)
#         rld = deepcopy(config_space_dict)
        rd, rdd = {},{}
        
        for ri,rule in enumerate(conv_rules,1):
    
            if (len(conv_rules) == 1) & (max_c[ri-1] == config_space.get_hyperparameter(rule_name).upper):
                break
            elif ri != len(conv_rules):
                rd[rule.name] = CS.ForbiddenEqualsClause(rule, max_c[ri-1])
            else:
                rd[rule.name] = CS.ForbiddenInClause(rule, range(max_c[ri-1] + 1, rule.upper + 1))
        
#         for ri,rulekey in enumerate(conv_dict_rules.keys(),1):
#             rule = conv_dict_rules[rulekey]
            
#             if (len(conv_dict_rules) == 1) & (max_c[ri-1] == config_space_dict[rule_name].upper):
#                 break
#             elif ri != len(conv_dict_rules):
#                 rld["%s_cond" % rulekey] = tune.randint(lower=0, upper=max_c[ri-1])
#                 rld["%s_cond" % rulekey] = tune.sample_from(lambda spec: spec.config_space_dict...uniform * 0.01)
#             else:
#                 rld[rulekey].upper = CS.ForbiddenInClause(rule, range(max_c[ri-1] + 1, rule.upper + 1))
        
        # package banlist for addition to config space
        # add banlist to collection
        if rd.values():
            config_space.add_forbidden_clause(
                CS.ForbiddenAndConjunction(
                    *rd.values()
                )
            )           
    
    for subindex in range(nodes_f):
        # define hyperparameter reference attributes
        rule_name = "full_subindex_%s" % subindex
        full_rule = CS.UniformIntegerHyperparameter(rule_name, lower=0, upper=base_f-1, default_value=subindex%(base_f-1))
        
        # add hyperparameter to collections
        config_space.add_hyperparameter(full_rule)
    
        full_rules = list(filter(lambda hp: "full_subindex_" in hp.name, config_space.get_hyperparameters()))
    
        # build banlist from collections
        rl = deepcopy(config_space)
        rd = {}
        for ri,rule in enumerate(full_rules,1):
            if (len(full_rules) == 1) & (max_f[ri-1] == config_space.get_hyperparameter(rule_name).upper):
                break
            elif ri != len(full_rules):
                rd[rule.name] = CS.ForbiddenEqualsClause(rule, max_f[ri-1])
            else:
                rd[rule.name] = CS.ForbiddenInClause(
                        rule,
                        range(max_f[ri-1] + 1, rule.upper + 1))
        # package banlist for addition to config space
        # add banlist to collection
        if rd.values():
            config_space.add_forbidden_clause(
                CS.ForbiddenAndConjunction(
                    *rd.values()
                )
            )
    
    for subindex in range(nodes_m):
        # define hyperparameter reference attributes
        rule_name = "maxp_subindex_%s" % subindex
        
        maxp_rule = CS.UniformIntegerHyperparameter(rule_name, lower=0, upper=base_m-1, default_value=subindex%(base_m-1))
#         conv_dict_rule = tune.randint(lower=0, upper=base_c-1)
        
        # add hyperparameter to collections
        config_space.add_hyperparameter(maxp_rule)
#         config_space_dict[rule_name] = conv_dict_rule
#         config_space_dict["conv_subindices"].append(conv_dict_rule)
    
        maxp_rules = list(filter(lambda hp: "maxp_subindex_" in hp.name, config_space.get_hyperparameters()))
        maxp_dict_rules_keys = list(filter(lambda name: "maxp_subindex_" in name, config_space_dict.keys()))
#         conv_dict_rules = {k:config_space_dict[k] for k in conv_dict_rules_keys if k in config_space_dict}
    
        # build banlist from collections
        rl = deepcopy(config_space)
#         rld = deepcopy(config_space_dict)
        rd, rdd = {},{}
        
        for ri,rule in enumerate(maxp_rules,1):
    
            if (len(maxp_rules) == 1) & (max_m[ri-1] == config_space.get_hyperparameter(rule_name).upper):
                break
            elif ri != len(maxp_rules):
                rd[rule.name] = CS.ForbiddenEqualsClause(rule, max_m[ri-1])
            else:
                rd[rule.name] = CS.ForbiddenInClause(rule, range(max_m[ri-1] + 1, rule.upper + 1))
        
#         for ri,rulekey in enumerate(conv_dict_rules.keys(),1):
#             rule = conv_dict_rules[rulekey]
            
#             if (len(conv_dict_rules) == 1) & (max_c[ri-1] == config_space_dict[rule_name].upper):
#                 break
#             elif ri != len(conv_dict_rules):
#                 rld["%s_cond" % rulekey] = tune.randint(lower=0, upper=max_c[ri-1])
#                 rld["%s_cond" % rulekey] = tune.sample_from(lambda spec: spec.config_space_dict...uniform * 0.01)
#             else:
#                 rld[rulekey].upper = CS.ForbiddenInClause(rule, range(max_c[ri-1] + 1, rule.upper + 1))
        
        # package banlist for addition to config space
        # add banlist to collection
        if rd.values():
            config_space.add_forbidden_clause(
                CS.ForbiddenAndConjunction(
                    *rd.values()
                )
            )           
    
    for subindex in range(nodes_d):
        # define hyperparameter reference attributes
        rule_name = "drop_subindex_%s" % subindex
        drop_rule = CS.UniformIntegerHyperparameter(rule_name, lower=0, upper=base_d-1, default_value=subindex%(base_d-1))
        
        # add hyperparameter to collections
        config_space.add_hyperparameter(drop_rule)
    
        drop_rules = list(filter(lambda hp: "drop_subindex_" in hp.name, config_space.get_hyperparameters()))
    
        # build banlist from collections
        rl = deepcopy(config_space)
        rd = {}
        for ri,rule in enumerate(drop_rules,1):
            if (len(drop_rules) == 1) & (max_d[ri-1] == config_space.get_hyperparameter(rule_name).upper):
                break
            elif ri != len(drop_rules):
                rd[rule.name] = CS.ForbiddenEqualsClause(rule, max_d[ri-1])
            else:
                rd[rule.name] = CS.ForbiddenInClause(
                        rule,
                        range(max_d[ri-1] + 1, rule.upper + 1))
        # package banlist for addition to config space
        # add banlist to collection
        if rd.values():
            config_space.add_forbidden_clause(
                CS.ForbiddenAndConjunction(
                    *rd.values()
                )
            )
        
    # add nodes to config space
    [config_space_dict["conv_subindices"].append(tune.randint(lower=0, upper=base_c-1)) for n in range(nodes_c)]
    [config_space_dict["maxp_subindices"].append(tune.randint(lower=0, upper=base_m-1)) for n in range(nodes_m)]
    [config_space_dict["full_subindices"].append(tune.randint(lower=0, upper=base_f-1)) for n in range(nodes_f)]
    [config_space_dict["drop_subindices"].append(tune.randint(lower=0, upper=base_d-1)) for n in range(nodes_d)]
    
    for i,ceiling in enumerate(max_c):
        # add cases wherein subindex must be switched to auxilliary
        if i:
            config_space_dict["conv_cases"].append(
                tune.sample_from(lambda spec: spec.config.conv_subindices[i] >= max_c[i] & config_space_dict["conv_cases"][i-1])
            )
        else:
            config_space_dict["conv_cases"].append(
                tune.sample_from(lambda spec: spec.config.conv_subindices[i] >= max_c[i])
            )
            
        # add auxilliary values
        config_space_dict["conv_auxilliary"].append(tune.randint(lower=0, upper=ceiling))
        
        # add subindex swticher
        config_space_dict["conv_adjusted"].append(
            tune.sample_from(lambda spec: 
                spec.config.conv_auxilliary[i] if spec.config.conv_cases[i] else spec.config.conv_subindices[i]
            )
        )
        
    for i,ceiling in enumerate(max_f):
        # add cases wherein subindex must be switched to auxilliary
        if i:
            config_space_dict["full_cases"].append(
                tune.sample_from(lambda spec: spec.config.full_subindices[i] >= max_f[i] & spec.config.full_cases[i-1])
            )
        else:
            config_space_dict["full_cases"].append(
                tune.sample_from(lambda spec: spec.config.full_subindices[i] >= max_f[i])
            )
            
        # add auxilliary values
        config_space_dict["full_auxilliary"].append(tune.randint(lower=0, upper=ceiling))
        
        # add subindex swticher
        config_space_dict["full_adjusted"].append(
            tune.sample_from(lambda spec: 
                spec.config.full_auxilliary[i] if spec.config.full_cases[i] else spec.config.full_subindices[i]
            )
        )
    
    for i,ceiling in enumerate(max_m):
        # add cases wherein subindex must be switched to auxilliary
        if i:
            config_space_dict["maxp_cases"].append(
                tune.sample_from(lambda spec: spec.config.maxp_subindices[i] >= max_m[i] & config_space_dict["maxp_cases"][i-1])
            )
        else:
            config_space_dict["maxp_cases"].append(
                tune.sample_from(lambda spec: spec.config.maxp_subindices[i] >= max_m[i])
            )
            
        # add auxilliary values
        config_space_dict["maxp_auxilliary"].append(tune.randint(lower=0, upper=ceiling))
        
        # add subindex swticher
        config_space_dict["maxp_adjusted"].append(
            tune.sample_from(lambda spec: 
                spec.config.maxp_auxilliary[i] if spec.config.maxp_cases[i] else spec.config.maxp_subindices[i]
            )
        )
        
    for i,ceiling in enumerate(max_d):
        # add cases wherein subindex must be switched to auxilliary
        if i:
            config_space_dict["drop_cases"].append(
                tune.sample_from(lambda spec: spec.config.drop_subindices[i] >= max_d[i] & spec.config.drop_cases[i-1])
            )
        else:
            config_space_dict["drop_cases"].append(
                tune.sample_from(lambda spec: spec.config.drop_subindices[i] >= max_d[i])
            )
            
        # add auxilliary values
        config_space_dict["drop_auxilliary"].append(tune.randint(lower=0, upper=ceiling))
        
        # add subindex swticher
        config_space_dict["drop_adjusted"].append(
            tune.sample_from(lambda spec: 
                spec.config.drop_auxilliary[i] if spec.config.drop_cases[i] else spec.config.drop_subindices[i]
            )
        )
        
    return config_space,config_space_dict

In [ ]:
cn = configure_neurons()
cfl = [{v:cn[1][v]} for v in cn[1]]
[print(v,": ",cn[1][v]) for v in cn[1]]

In [ ]:
# perform neuron configuration trials
def search_neurons(checkpoint_dir=None):    
    neuron_config_space = configure_neurons()[1]
    parameter_columns = ["batch_size", "lr", "epochs", "conv_adjusted", "full_adjusted"]
    
    experiment_metrics = dict(metric="accuracy", mode="max")
    
#     config = {
#     "LiNO3_vol": tune.uniform(0, 7),
#     "Li2SO4_vol": tune.uniform(0, 7),
#     "NaClO4_vol": tune.uniform(0, 7)
#     }
    
#     #pre-load data to avoid races
#     load_data()
    
#     param_dict = {"domain": cfl}
#     domain_config = load_config(param_dict)
#     print(domain_config)
#     domain, domain_orderings = domain_config.domain, domain_config.domain_orderings

    # define the hpo search algorithm BO
#     func_caller = EuclideanFunctionCaller(None, domain_config.domain.list_of_domains[0])
#     optimizer = EuclideanGPBandit(func_caller, ask_tell_mode=True)
#     bo_search_alg = DragonflySearch(optimizer, **experiment_metrics)
    
    scheduler = PB2(
        time_attr="training_iteration",
        perturbation_interval=1,#max_time/10,
#         quantile_fraction=0.5,
        hyperparam_bounds=neuron_config_space,
        synch=True,
        **experiment_metrics)
#     search = DragonflySearch(
#         optimizer="bandit",
#         domain="euclidean",
# #         space=neuron_config_space,
# #         space=cfl,
# #         max_concurrent=8,
#         **experiment_metrics)
#     search = ConcurrencyLimiter(search, max_concurrent=8)
    reporter = JupyterNotebookReporter(
        overwrite=True,
#         parameter_columns=neuron_config_space.get_hyperparameter_names(),
        parameter_columns=parameter_columns,
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar),
        verbose=3,
        name="neurons",
        local_dir=r.absolute(),
        resources_per_trial={"cpu": cpu_use, "gpu": gpu_use},
        max_failures=3,
        num_samples=num_samples,
        scheduler=scheduler,
        search_alg=search,
        config=param_dict,
        queue_trials=True,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("accuracy", "max", "last")
    
    def cv_discrim(s): return 'conv_subindex_' in s
    def fc_discrim(s): return 'full_subindex_' in s
    def other_discrim(s): return 'subindex' not in s
    best_cvs = [best_trial.config[hp] for hp in list(filter(cv_discrim, best_trial.config.keys()))]
    best_fcs = [best_trial.config[hp] for hp in list(filter(fc_discrim, best_trial.config.keys()))]
    best_other = [best_trial.config[hp] for hp in list(filter(other_discrim, best_trial.config.keys()))]

    cfg = decode([best_cvs, best_fcs])
    
    conv_report = ["Connolutional Layer %s: %s" % (i,c) for i,c in enumerate(cfg[0],1)]
    full_report = ["Fully-connected Layer %s: %s" % (i,f) for i,f in enumerate(cfg[1],1)]
    other_report = ["%s: %s" % (hp,f) for (hp,f) in zip(["Batch Size","Epochs","Learning Rate"],best_other)]

#     print("Best trial config: {}".format(best_trial.config))
    print("Best trial config:")
    [print(best) for best in [conv_report,full_report,other_report]]
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))
    
    best_trained_model = Net(cfg)
    best_training_hyperparameters = zip(["Batch Size","Epochs","Learning Rate"],best_other)
    
    if gpus_per_trial > 1:
        best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    
    if checkpoint_dir != None:
        tune.report(accuracy=test_acc)
    
    print("Best trial test set accuracy: {}".format(test_acc))
    
    return (best_trained_model.state_dict(), dict(best_training_hyperparameters))


# perform test
if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    model = search_layers(num_samples=10, max_num_epochs=10, gpus_per_trial=1)


In [ ]:
print("Resource usage can be viewed at port http://127.0.0.1:8265/ or higher")

In [ ]:
model,trainers = search_neurons()

In [ ]:
print(model)